<a href="https://colab.research.google.com/github/subraja1810/BERT/blob/main/CS18B1059.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2


     |████████████████████████████████| 51kB 4.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=0b426ad89bad3eacd67ef2dc6ff7c7a4aee87ac48e23f57ad332385b70747fee
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=ea8a189e6c4924776a3c131a0161fe8c3b60649a4867fe70fad4dc1a0bdfe2f9
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=76b305175787178f21a9f245f3bb7b19161eeecc53e333080d32cfec7391401d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
!pip install transformers
!pip install praw

     |████████████████████████████████| 1.3MB 11.9MB/s 
     |████████████████████████████████| 2.9MB 39.5MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 1.1MB 34.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1da94c6b1c5a1699b335550ee1ea147a370fbc49456ad2f32d752b9c1a9d239a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 153kB 12.1MB/s 
     |████████████████████████████████| 204kB 19.5MB/s 


In [4]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer 
from matplotlib import pyplot as plt 
import praw 
import pandas as pd 

In [5]:
model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english") 
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english and are newly initialized: ['dropout_73']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity immediately after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name immediately after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation immediately after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location immediately after another location
    "I-LOC"    # Location
]

In [54]:
reddit = praw.Reddit(client_id='3PQwAJT-jv_t2g', 
                     client_secret='Lj2yqyiO-r5RO-K-7NAM7ABx9mL4qg',
                     user_agent='RedditBoo')

In [55]:
def replies_of(top_comments, comment_list):
    if len(top_comments.replies) == 0:
        return
    else:
        for num, comment in enumerate(top_comments.replies):
            try:
                comments_list.append(str(comment.body))
            except:
                continue
            replies_of(comment, comments_list)

In [ ]:
count = 0
final_dict = {'I-LOC': [], 'I-ORG': [], 'I-PER': [], 'B-LOC': [], 'B-ORG': [], 'B-PER': [], 'B-MISC': [], 'I-MISC': []} 
word_temp = ''
current_tag = ''
old_tag = ''
print_dict = {}
print(final_dict)

In [57]:
list_of_subreddit = ['marvelstudios']

In [ ]:
for j in list_of_subreddit:
    top_posts = reddit.subreddit(j).top('month', limit=1)
    comments_list = [] #Data fetched.
    # save subreddit comments in dataframe
    for submission in top_posts:
        submission_comm = reddit.submission(id=submission.id)
        comments_list.append(str(submission.title))

        for count, top_comments in enumerate(submission_comm.comments):
            try:
                replies_of(top_comments, comments_list)
            except:
                continue

print(comments_list)                

In [65]:
for sequence in comments_list:
    if len(sequence) > 512: #BERT token limitaton.
        continue
    # Standard Tokenization
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="tf")
    outputs = model(inputs)[0]
    predictions = tf.argmax(outputs, axis=2)
   # print(predictions.shape)
    # Tokenized input
    list_bert = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]
    for i in list_bert:
        if i[1] in ['O']:
            if len(current_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    final_dict[current_tag].append(without_space_word)
            count = 0
            word_temp = ''
            current_tag = ''
            continue
        else:
            current_tag = i[1]

            if old_tag != current_tag and len(old_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    final_dict[old_tag].append(without_space_word)
                count = 0
                word_temp = ''
                current_tag = ''

            if i[0].startswith('##'): 
                word_temp += i[0][2:].upper()
            elif i[1] in ['I-PER', 'I-ORG', 'I-LOC', 'B-LOC', 'B-ORG', 'B-PER', 'B-MISC', 'I-MISC']:
                word_temp += " " + i[0].upper()
                current_tag = i[1]
                count += 1
            old_tag = current_tag

In [86]:
print(final_dict)
print_dict['Location']=list(set(final_dict['I-LOC']+final_dict['B-LOC']))
print_dict['Organisation']=list(set(final_dict['I-ORG']+final_dict['B-ORG']))
print_dict['Person Name']=list(set(final_dict['I-PER']+final_dict['B-PER']))
print_dict['Miscellaneous']=list(set(final_dict['I-MISC']+final_dict['B-MISC']))
print('')
print(print_dict)

{'I-LOC': ['WAKANDA', 'WAKANDA', 'WAKANDA', 'NYC', 'WAKANDA', 'EUROPE', 'US', 'HOLLYWOOD', 'NEUSCHWANSTEIN', 'CHINA', 'TIBET', 'CHINA', 'TIBET', 'CHINA', 'CHINA', 'CHINA', 'CHINA', 'CHINA', 'HOLLYWOOD', 'CHINA', 'CHINA', 'CHINA', 'CHINA', 'CHINA', 'NEW YORK', 'CHINA', 'CHINA', 'CHINA', 'WAKANDA', 'HOLLYWOOD', 'THANOS'], 'I-ORG': ['WANDAVISION', 'BOSEMAN MCU', 'MARVEL', 'MCU', 'MARVEL', 'WANDAVISION', 'PANTHER', 'WIDOW', 'WANDAVISION', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDAFOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'WAKANDA FOREVER', 'YI', 'YI', 'YI', 'YI', 'YI', 'YI', 'YI', 'YI', 'GUILLA', 'MARVEL', 'MCU', 'MCU', 'MARVEL', 'MCU', 'MCU', 'BLACK PANTHER', 'AVENGERS', 'MCU', 'MARVEL', 'DISNEY', 'MARVEL', 'WB', 'DC', 'DC', 'MCU', 'MCU', 'DC', 'MARVEL', 'MARVEL', 'DC', 'FL

In [88]:
list_of_loc = final_dict['I-LOC']
#list_of_org = final_dict['I-ORG']
#list_of_person = final_dict['I-PER']
#list_of_misc = final_dict['I-MISC']
frequency = {}
for loc in list_of_loc:
  frequency[loc] = frequency.get(loc, 0) +1
  print(frequency)

{'WAKANDA': 1}
{'WAKANDA': 2}
{'WAKANDA': 3}
{'WAKANDA': 3, 'NYC': 1}
{'WAKANDA': 4, 'NYC': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 1, 'TIBET': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 2, 'TIBET': 1}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 2, 'TIBET': 2}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 3, 'TIBET': 2}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1, 'US': 1, 'HOLLYWOOD': 1, 'NEUSCHWANSTEIN': 1, 'CHINA': 4, 'TIBET': 2}
{'WAKANDA': 4, 'NYC': 1, 'EUROPE': 1,

In [ ]:
import pandas as pd
data = {'Location': list_of_loc,
        'Count': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
        }

df = pd.DataFrame(data, columns = ['Location', 'Count'])

print (df)

In [40]:
#print(master_dict)

{'I-LOC': ['CHENNAI', 'CHENNAI', 'INDIA', 'CHENNAI', 'BANGLORE'], 'I-ORG': ['TECHNOFORTE'], 'I-PER': ['AJAY', 'AJAY', 'AJAY', 'SUBRAJA', 'SUBRAJA'], 'B-LOC': [], 'B-ORG': [], 'B-PER': []}


In [37]:
#comment_list=['Hi, I am ajay and I am from India. I work for Technoforte located in Chennai.', 
#'I am a huge fan of marvel and my favourite character is Subraja. Subraja is from Banglore' ]

In [33]:
#outputs[0][6]

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([-0.7641342 , -2.4077516 , -2.0843105 , -3.2465644 ,  0.10769157,
       -2.4875414 ,  1.225174  , -1.9893391 ,  7.9986925 ], dtype=float32)>

In [34]:
#label_list[8]

'I-LOC'

In [35]:
#sequence,tokens

('I am Ajay from Chennai',
 ['[CLS]', 'I', 'am', 'A', '##jay', 'from', 'Chennai', '[SEP]'])

In [36]:
#predictions.numpy()[0]

array([0, 0, 0, 4, 4, 0, 8, 0])